In [1]:
# 加载数据
import pandas as pd

# mml系统数据
# v1
# history_df = pd.read_excel(r'准备数据\MML系统数据\Histories (2022.08.05 08.30.51 AM GMT+00_00).xlsx')
# assessment_df = pd.read_excel(r'准备数据\MML系统数据\Resolved Assessments _ Tests (AUSTRALIA)  20220805.xlsx', sheet_name=1)
# v2
access_log_df = pd.read_excel(r'准备数据\MML系统数据\Access Log - AU and IN - March to Aug - 20220811.xlsx')    # 澳洲和印度市场登录日志
wb0039_complement_df = pd.read_excel(r'准备数据\MML系统数据\Competencies - Sales Learning Programme - AU - (2022.08.11 08.31.42 AM GMT+00_00).xlsx', skiprows=lambda x : x in list(range(6)) + [7, 8])   # 澳洲市场WB0039等课程完成情况
assessment_df = pd.read_excel(r'准备数据\MML系统数据\Resolved Assessments _ Tests (INDIA) (1).xlsx', sheet_name=1)   # 印度市场测验成绩
course_rating_IN_df = pd.read_excel(r'准备数据\MML系统数据\Resolved Course Ratings (INDIA).xlsx', sheet_name=1, skiprows=[1])   # 印度市场课程评分
course_rating_AU_df = pd.read_excel(r'准备数据\MML系统数据\Resolved Course Ratings (AUSTRALIA) (1).xlsx', sheet_name=1, skiprows=[1])     # 澳洲市场课程评分
wb0998_complement_df = pd.read_excel(r'准备数据\MML系统数据\Training Map - Sales Learning Programme - India (2022.08.11 08.30.12 AM GMT+00_00).xlsx', skiprows=lambda x : x in list(range(6)) + [7])   # 印度市场WB0998等课程完成情况
wb0982_complement_df = pd.read_excel(r'准备数据\MML系统数据\Training Map VCC5040 - AU and IN - (2022.08.11 08.26.42 AM GMT+00_00).xlsx', skiprows=lambda x : x in list(range(6)) + [7])   # 澳洲和印度市场WB0982等课程完成情况


# sales cloud系统数据
lead_df = pd.read_csv(r'准备数据\Sales Could系统数据\lead.csv')
opportunity_df = pd.read_csv(r'准备数据\Sales Could系统数据\opportunity.csv')
opportunityid_ownerid_df = pd.read_csv(r'准备数据\Sales Could系统数据\opportunityid_ownerid.csv')
order_df = pd.read_csv(r'准备数据\Sales Could系统数据\order.csv')
test_drive_df = pd.read_csv(r'准备数据\Sales Could系统数据\testdrive.csv')
user_info_df = pd.read_csv(r'准备数据\Sales Could系统数据\user_info.csv')

C:\Users\Riddance\AppData\Local\Temp\ipykernel_29744\3736563322.py:18: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  lead_df = pd.read_csv(r'准备数据\Sales Could系统数据\lead.csv')


In [16]:
# 按经销商维度统计课程评分数量
course_rating_df = pd.concat([course_rating_IN_df, course_rating_AU_df], axis=0)    # 合并印度和澳洲市场课程评分
CR_D_COUNT = course_rating_df.groupby(['DEALER CODE'])['AVERAGE'].count()
CR_D_COUNT.rename('QTY', inplace=True)
CR_D_COUNT_df = CR_D_COUNT.reset_index()
CR_D_COUNT_df = pd.merge(CR_D_COUNT_df, course_rating_df, on='DEALER CODE')[['DEALER CODE','DEALER','QTY']]
CR_D_COUNT_df.drop_duplicates(inplace=True)
CR_D_COUNT_df.sort_values(inplace=True, by=['DEALER'])

In [17]:
# 按经销商维度统计课程完成数量绘图
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode
 
init_notebook_mode(connected=True)

if not os.path.exists('figure'):
    os.mkdir('figure')

fig = go.Figure()
fig = px.scatter(CR_D_COUNT_df, 
                 x='DEALER', 
                 y='QTY',
                 color='QTY',
                 size='QTY')
fig.update_layout(title='QTY of Course Rating by Dealer',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/CR_D_COUNT.png')
fig.show()

In [20]:
# 按经销商维度统计课程评分标准差
SD_CR_D = course_rating_df.groupby('DEALER CODE')['AVERAGE'].std().round(2)
SD_CR_D.rename('SD_CR_D', inplace=True)
SD_CR_D_df = SD_CR_D.reset_index()
SD_CR_D_df = pd.merge(SD_CR_D_df, course_rating_df, on='DEALER CODE')[['DEALER CODE','DEALER','SD_CR_D']]
SD_CR_D.drop_duplicates(inplace=True)
SD_CR_D_df.sort_values(inplace=True, by=['DEALER'])

In [29]:
# 按经销商维度统计课程评分标准差绘图 
SD_CR_D_df.dropna(inplace=True)
SD_CR_D_df['size'] = SD_CR_D_df['SD_CR_D'] + 1

fig = go.Figure()
fig = px.scatter(SD_CR_D_df, 
                 x='DEALER', 
                 y='SD_CR_D',
                 color='SD_CR_D',
                 size='size')
fig.update_layout(title='STD of Course Rating by Dealer',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_CR_D.png')
fig.show()

In [31]:
# 按市场维度统计课程评分标准差
CR_M_COUNT = course_rating_df.groupby(['MARKET'])['AVERAGE'].count()
CR_M_COUNT.rename('COUNT', inplace=True)
SD_CR_M = course_rating_df.groupby('MARKET')['AVERAGE'].std().round(2)
SD_CR_M.rename('SD_CR_M', inplace=True)

MARKET
AUSTRALIA    1.53
INDIA        0.87
Name: SD_CR_M, dtype: float64

In [34]:
# 按市场维度统计课程评分标准差绘图
SD_CR_M_df = SD_CR_M.reset_index() 
SD_CR_M_df.dropna(inplace=True)
SD_CR_M_df['size'] = SD_CR_M_df['SD_CR_M'] + 1

fig = go.Figure()
fig = px.scatter(SD_CR_M_df, 
                 x='MARKET', 
                 y='SD_CR_M',
                 color='SD_CR_M',
                 size='size')
fig.update_layout(title='STD of Course Rating by Market',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_CR_M.png')
fig.show()

In [35]:
from openpyxl import load_workbook
excel_file = '准备数据\MML系统数据\Training Map - Sales Learning Programme - India (2022.08.11 08.30.12 AM GMT+00_00).xlsx' 
wb = load_workbook(excel_file, data_only = True)
sh = wb['Competencies']

# 获取状态颜色字典
color_in_hex = [cell.fill.start_color.index for cell in sh['AO:AO'][:6]]
status = ['COMPLETED', 'KNOWLEDGE GAP IDENTIFIED', 'IN PROGRESS', 'ENROLLED', 'ABSENT', 'NOT STARTED']
status_color_dict = dict(zip(status, color_in_hex))

# 统计完成课程数量
def cal_complete_lecture(min_row, min_col, sh, df):
    index = 0
    for row in sh.iter_rows(min_row=min_row, max_row=sh.max_row, min_col=min_col, max_col=sh.max_column):
        all_lecture = 0
        complete_lecture = 0
        for cell in row:
            if cell.fill.start_color.index == status_color_dict['COMPLETED']:
                complete_lecture += 1
            all_lecture += 1
        df.loc[index, 'COMPLETED'] = complete_lecture
        df.loc[index, 'ALL'] = all_lecture
        index += 1
    return df

# 统计wb0039课程完成数量
min_row = 9
min_col = 13
wb0998_complement_df = cal_complete_lecture(min_row, min_col, sh, wb0998_complement_df)

# 统计wb0982课程完成数量
excel_file = '准备数据\MML系统数据\Training Map VCC5040 - AU and IN - (2022.08.11 08.26.42 AM GMT+00_00).xlsx'
wb = load_workbook(excel_file, data_only = True)
sh = wb['Competencies']
min_row = 9
min_col = 13
wb0982_complement_df = cal_complete_lecture(min_row, min_col, sh, wb0982_complement_df)

# 统计wb0039课程完成数量
excel_file = '准备数据\MML系统数据\Competencies - Sales Learning Programme - AU - (2022.08.11 08.31.42 AM GMT+00_00).xlsx'
wb = load_workbook(excel_file, data_only = True)
sh = wb['1 Sales (previously Consultativ']
min_row = 10
min_col = 16
wb0039_complement_df = cal_complete_lecture(min_row, min_col, sh, wb0039_complement_df)

In [45]:
# 按经销商维度统计课程完成数量标准差
complement_df = pd.concat([wb0998_complement_df, wb0982_complement_df, wb0039_complement_df], axis=0)
CC_D_COUNT = complement_df.groupby(['DEALER CODE'])['COMPLETED'].count()
CC_D_COUNT.rename('COUNT', inplace=True)
SD_CC_D = complement_df.groupby('DEALER CODE')['COMPLETED'].std().round(2)
SD_CC_D.rename('SD_CC_D', inplace=True)
SD_CC_D_df = SD_CC_D.reset_index()
SD_CC_D_df = pd.merge(SD_CC_D_df, complement_df, on='DEALER CODE')[['DEALER CODE','DEALER','SD_CC_D']]
SD_CC_D_df.drop_duplicates('DEALER CODE', inplace=True)
SD_CC_D_df.sort_values(inplace=True, by=['DEALER'])

In [47]:
# 按经销商维度统计课程完成数量标准差绘图
SD_CC_D_df.dropna(inplace=True)
SD_CC_D_df['size'] = SD_CC_D_df['SD_CC_D'] + 1

fig = go.Figure()
fig = px.scatter(SD_CC_D_df, 
                 x='DEALER', 
                 y='SD_CC_D',
                 color='SD_CC_D',
                 size='size')
fig.update_layout(title='STD of Course Completed by Dealer',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_CC_D.png')
fig.show()

In [49]:
# 按市场维度统计课程完成数量标准差
CC_M_COUNT = complement_df.groupby(['MARKET'])['COMPLETED'].count()
CC_M_COUNT.rename('COUNT', inplace=True)
SD_CC_M = complement_df.groupby('MARKET')['COMPLETED'].std().round(2)
SD_CC_M.rename('SD_CC_M', inplace=True)

MARKET
AUSTRALIA     8.10
INDIA        11.51
Name: SD_CC_M, dtype: float64

In [51]:
# 按市场维度统计课程完成数量标准差绘图
SD_CC_M_df = SD_CC_M.reset_index() 
SD_CC_M_df.dropna(inplace=True)
SD_CC_M_df['size'] = SD_CC_M_df['SD_CC_M'] + 1

fig = go.Figure()
fig = px.scatter(SD_CC_M_df, 
                 x='MARKET', 
                 y='SD_CC_M',
                 color='SD_CC_M',
                 size='size')
fig.update_layout(title='STD of Course Completed by Market',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_CC_M.png')
fig.show()

In [54]:
assessment_df.dropna(how='all', inplace=True)

# 按经销商维度统计课程测验分数标准差
CPT_D_COUNT = assessment_df.groupby(['DEALER CODE'])['GRADE'].count()
CPT_D_COUNT.rename('COUNT', inplace=True)
SD_CPT_D = assessment_df.groupby('DEALER CODE')['GRADE'].std().round(2)
SD_CPT_D.rename('SD_CPT_D', inplace=True)
SD_CPT_D_df = SD_CPT_D.reset_index()
SD_CPT_D_df = pd.merge(SD_CPT_D_df, assessment_df, on='DEALER CODE')[['DEALER CODE','DEALER','SD_CPT_D']]
SD_CPT_D_df.drop_duplicates('DEALER CODE', inplace=True)
SD_CPT_D_df.sort_values(inplace=True, by=['DEALER'])

In [53]:
# 按经销商维度统计课程完成数量标准差绘图
SD_CPT_D_df.dropna(inplace=True)
SD_CPT_D_df['size'] = SD_CPT_D_df['SD_CPT_D'] + 1

fig = go.Figure()
fig = px.scatter(SD_CPT_D_df, 
                 x='DEALER', 
                 y='SD_CPT_D',
                 color='SD_CPT_D',
                 size='size')
fig.update_layout(title='STD of Course Post-Test Score by Dealer',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_CPT_D.png')
fig.show()

In [55]:
# 按市场维度统计课程测验分数标准差
CC_M_COUNT = assessment_df.groupby(['MARKET'])['GRADE'].count()
CC_M_COUNT.rename('COUNT', inplace=True)
SD_CPT_M = assessment_df.groupby('MARKET')['GRADE'].std().round(2)
SD_CPT_M.rename('SD_CPT_M', inplace=True)

MARKET
INDIA    11.34
Name: SD_CPT_M, dtype: float64

In [57]:
# 按市场维度统计课程完成数量标准差绘图
SD_CPT_M_df = SD_CPT_M.reset_index() 
SD_CPT_M_df.dropna(inplace=True)
SD_CPT_M_df['size'] = SD_CPT_M_df['SD_CPT_M'] + 1

fig = go.Figure()
fig = px.scatter(SD_CPT_M_df, 
                 x='MARKET', 
                 y='SD_CPT_M',
                 color='SD_CPT_M',
                 size='size')
fig.update_layout(title='STD of Course Post-Testg Score by Market',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_CPT_M.png')
fig.show()

In [58]:
# 按经销商维度统计课程完成率标准差
complement_df['RATE'] = round(complement_df['COMPLETED'] / complement_df['ALL'] * 100, 2)
SLPC_D_COUNT = complement_df.groupby(['DEALER CODE'])['RATE'].count()
SLPC_D_COUNT.rename('COUNT', inplace=True)
SD_SLPC_D = complement_df.groupby('DEALER CODE')['RATE'].std()
SD_SLPC_D.rename('SD_SLPC_D', inplace=True)
SD_SLPC_D_df = SD_SLPC_D.reset_index()
SD_SLPC_D_df = pd.merge(SD_SLPC_D_df, complement_df, on='DEALER CODE')[['DEALER CODE','DEALER','SD_SLPC_D']]
SD_SLPC_D_df.drop_duplicates('DEALER CODE', inplace=True)
SD_SLPC_D_df.sort_values(inplace=True, by=['DEALER'])

In [60]:
# 按经销商维度统计课程完成率标准差绘图
SD_SLPC_D_df.dropna(inplace=True)
SD_SLPC_D_df['size'] = SD_SLPC_D_df['SD_SLPC_D'] + 1

fig = go.Figure()
fig = px.scatter(SD_SLPC_D_df, 
                 x='DEALER', 
                 y='SD_SLPC_D',
                 color='SD_SLPC_D',
                 size='size')
fig.update_layout(title='STD of Sales Learning Programme Completion(%) by Dealer',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_SLPC_D.png')
fig.show()

In [61]:
# 按市场维度统计课程完成率标准差
SLPC_M_COUNT = complement_df.groupby(['MARKET'])['RATE'].count()
SLPC_M_COUNT.rename('COUNT', inplace=True)
SD_SLPC_M = complement_df.groupby('MARKET')['RATE'].std()
SD_SLPC_M.rename('SD_SLPC_M', inplace=True)

MARKET
AUSTRALIA    41.710225
INDIA        38.917082
Name: SD_SLPC_M, dtype: float64

In [62]:
# 按市场维度统计课程完成率标准差绘图
SD_SLPC_M_df = SD_SLPC_M.reset_index() 
SD_SLPC_M_df.dropna(inplace=True)
SD_SLPC_M_df['size'] = SD_SLPC_M_df['SD_SLPC_M'] + 1

fig = go.Figure()
fig = px.scatter(SD_SLPC_M_df, 
                 x='MARKET', 
                 y='SD_SLPC_M',
                 color='SD_SLPC_M',
                 size='size')
fig.update_layout(title='STD of Sales Learning Programme Completion(%) by Market',
                  title_font_size=20,
                  template='plotly_white',)
fig.write_image(r'figure/SD_SLPC_M.png')
fig.show()

In [70]:
def plot(df, name, title, by):
    df = df.reset_index() 
    df.dropna(inplace=True)
    df['size'] = df[name] + 1

    fig = go.Figure()
    fig = px.scatter(df, 
                    x=by, 
                    y=name,
                    color=name,
                    size='size')
    fig.update_layout(title=title,
                    title_font_size=20,
                    template='plotly_white',)
    fig.write_image(rf'figure/{name}.png')
    fig.show()

In [72]:
# 按市场维度统计潜在客户数量标准差

# 按经销商维度统计潜在客户数量
OP_D_COUNT = opportunity_df.groupby(['vccmarket__c', 'partnerid__c']).count()['id']
OP_D_COUNT.rename('COUNT', inplace=True)
OP_D_COUNT_df = OP_D_COUNT.reset_index()

# 按市场维度统计潜在客户数量标准差
SD_OP_M = OP_D_COUNT_df.groupby(['vccmarket__c'])['COUNT'].std().round(2)
SD_OP_M.rename('SD_OP_M', inplace=True)
SD_OP_M_df = SD_OP_M.reset_index()
SD_OP_M_df.rename(columns={'vccmarket__c':'MARKET'}, inplace=True)

In [73]:
# 按市场维度统计潜在客户数量标准差绘图
df = SD_OP_M_df
name = 'SD_OP_M'
title = 'STD of Opportunity by Market'
by = 'MARKET'
plot(df, name, title, by)

In [63]:
# 按经销商维度统计潜在客户数量标准差
pd.merge(opportunity_df, opportunityid_ownerid_df, left_on='id', right_on='Id', how='left')

,id,partnerid__c,name__c,dedealergroupcode,dealergroupname,createddate,leadtoopportunitytime__c,leadtoopportunityduration__c,vccmarket__c,Id,OwnerId,attributes.type,attributes.url
0,0062w00000EsynuAAB,6IN6038,"Kerala Volvo, Kochi",NaN,Kerala,2022-08-08T06:22:05.000+0000,2022-08-08T06:30:42.000+0000,0.0,IN,NaN,NaN,NaN,NaN
1,0062w00000EsbihAAB,6IN6038,"Kerala Volvo, Kochi",NaN,Kerala,2022-08-03T13:08:22.000+0000,2022-08-03T13:15:59.000+0000,0.0,IN,NaN,NaN,NaN,NaN
2,0062w00000EsWPuAAN,6IN6038,"Kerala Volvo, Kochi",NaN,Kerala,2022-08-03T05:03:10.000+0000,2022-08-03T10:37:55.000+0000,6.0,IN,NaN,NaN,NaN,NaN
3,0062w00000EsQLNAA3,6IN6038,"Kerala Volvo, Kochi",NaN,Kerala,2022-08-02T05:10:54.000+0000,2022-08-06T05:45:36.000+0000,97.0,IN,NaN,NaN,NaN,NaN
4,0062w00000EsP4LAAV,6IN6038,"Kerala Volvo, Kochi",NaN,Kerala,2022-08-01T13:41:35.000+0000,2022-08-02T13:36:53.000+0000,24.0,IN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53607,0062w00000Ds6QUAAZ,6AU4875,Volvo Cars Cairns,NaN,NaN,2022-06-10T00:22:04.000+0000,2022-06-10T00:30:41.000+0000,0.0,AU,NaN,NaN,NaN,NaN
53608,0062w00000DZiTaAAL,6AU4875,Volvo Cars Cairns,NaN,NaN,2022-05-24T05:52:48.000+0000,2022-05-24T05:54:41.000+0000,0.0,AU,NaN,NaN,NaN,NaN
53609,0062w00000DZvHwAAL,6AU4875,Volvo Cars Cairns,NaN,NaN,2022-05-26T06:39:36.000+0000,2022-05-26T06:42:51.000+0000,0.0,AU,NaN,NaN,NaN,NaN
53610,0062w00000DZCkjAAH,6AU4875,Volvo Cars Cairns,NaN,NaN,2022-05-19T04:00:30.000+0000,2022-05-19T04:01:36.000+0000,0.0,AU,NaN,NaN,NaN,NaN


In [74]:
# 按市场维度统计试驾数量标准差

# 按经销商维度统计试驾数量
TD_D_COUNT = test_drive_df.groupby(['vccmarket__c', 'partnerid__c']).count()['id']
TD_D_COUNT.rename('COUNT', inplace=True)
TD_D_COUNT_df = TD_D_COUNT.reset_index()

# 按市场维度统计试驾数量标准差
SD_TD_M = TD_D_COUNT_df.groupby(['vccmarket__c'])['COUNT'].std().round(2)
SD_TD_M.rename('SD_TD_M', inplace=True)
SD_TD_M_df = SD_TD_M.reset_index()
SD_TD_M_df.rename(columns={'vccmarket__c':'MARKET'}, inplace=True)

In [75]:
# 按市场维度统计试驾数量标准差绘图
df = SD_TD_M_df
name = 'SD_TD_M'
title = 'STD of Test Drives by Market'
by = 'MARKET'
plot(df, name, title, by)

In [78]:
# 按市场维度统计订单数量标准差

# 按经销商维度统计订单数量
RO_D_COUNT = order_df.groupby(['vccmarket__c', 'partnerid__c']).count()['id']
RO_D_COUNT.rename('COUNT', inplace=True)
RO_D_COUNT_df = RO_D_COUNT.reset_index()

# 按市场维度统计订单数量标准差
SD_RO_M = RO_D_COUNT_df.groupby(['vccmarket__c'])['COUNT'].std().round(2)
SD_RO_M.rename('SD_RO_M', inplace=True)
SD_RO_M_df = SD_RO_M.reset_index()
SD_RO_M_df.rename(columns={'vccmarket__c':'MARKET'}, inplace=True)

In [79]:
# 按市场维度统计订单数量标准差绘图
df = SD_RO_M_df
name = 'SD_RO_M'
title = 'STD of Retail Orders by Market'
by = 'MARKET'
plot(df, name, title, by)

In [87]:
# 按市场维度统计潜在客户-试驾转化率标准差

# 按经销商维度统计转化率
OP_TD_df = pd.merge(OP_D_COUNT_df, TD_D_COUNT_df, left_on='partnerid__c', right_on='partnerid__c', how='inner')
OP_TD_df['RATE'] = round(OP_TD_df['COUNT_y'] / OP_TD_df['COUNT_x'] * 100, 2)

# 按市场维度统计转化率标准差
SD_OPTDR_M = OP_TD_df.groupby(['vccmarket__c_x'])['RATE'].std().round(2)
SD_OPTDR_M.rename('SD_OPTDR_M', inplace=True)
SD_OPTDR_M_df = SD_OPTDR_M.reset_index()
SD_OPTDR_M_df.rename(columns={'vccmarket__c_x':'MARKET'}, inplace=True)

In [88]:
# 按市场维度统计潜在客户-试驾转化率标准差绘图
df = SD_OPTDR_M_df
name = 'SD_OPTDR_M'
title = 'STD of Opportunity to Test Drive Rate by Market'
by = 'MARKET'
plot(df, name, title, by)

In [96]:
# 按市场维度统计潜在客户-订单转化率标准差

# 按经销商维度统计转化率
OP_RO_df = pd.merge(OP_D_COUNT_df, RO_D_COUNT_df, left_on='partnerid__c', right_on='partnerid__c', how='inner')
OP_RO_df['RATE'] = round(OP_RO_df['COUNT_y'] / OP_RO_df['COUNT_x'] * 100, 2)

# 按市场维度统计转化率标准差
SD_OPROR_M = OP_RO_df.groupby(['vccmarket__c_x'])['RATE'].std().round(2)
SD_OPROR_M.rename('SD_OPROR_M', inplace=True)
SD_OPROR_M_df = SD_OPROR_M.reset_index()
SD_OPROR_M_df.rename(columns={'vccmarket__c_x':'MARKET'}, inplace=True)

In [97]:
# 按市场维度统计潜在客户-试驾转化率标准差绘图
df = SD_OPROR_M_df
name = 'SD_OPROR_M'
title = 'STD of opportunity-to-retail-order-rates Test Drive Rate by Market'
by = 'MARKET'
plot(df, name, title, by)

In [109]:
# 计算潜在客户-订单成交销售周期
OP_OR_df = pd.merge(opportunity_df, order_df, left_on='id', right_on='opportunity__c', how='inner')
OP_OR_df = OP_OR_df[OP_OR_df['status__c'] == 'Completed']
OP_OR_df['leadtoopportunitytime__c'] = pd.to_datetime(OP_OR_df['leadtoopportunitytime__c'])
OP_OR_df['leadtoopportunitytime__c'] = OP_OR_df['leadtoopportunitytime__c'].dt.tz_localize(None)
OP_OR_df['ordersubmitteddate__c'] = pd.to_datetime(OP_OR_df['ordersubmitteddate__c'])
OP_OR_df['ordersubmitteddate__c'] = OP_OR_df['ordersubmitteddate__c'].dt.tz_localize(None)
OP_OR_df['duration'] = (OP_OR_df['ordersubmitteddate__c'] - OP_OR_df['leadtoopportunitytime__c']).dt.days

In [111]:
# 按市场维度统计成交销售周期标准差
OP_OR_df = OP_OR_df[OP_OR_df['duration'] >= -1]
OP_OR_df['duration'] = OP_OR_df['duration'].apply(lambda x: x if x > 0 else 0)
SD_CD_M = OP_OR_df.groupby(['vccmarket__c_x'])['duration'].std().round(2)
SD_CD_M.rename('SD_CD_M', inplace=True)
SD_CD_M_df = SD_CD_M.reset_index()
SD_CD_M_df.rename(columns={'vccmarket__c_x':'MARKET'}, inplace=True)

In [112]:
# 按市场维度统计成交销售周期标准差绘图
df = SD_CD_M_df
name = 'SD_CD_M'
title = 'STD of cycle-duration by Market'
by = 'MARKET'
plot(df, name, title, by)